In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import uproot

from scipy import stats
from sklearn import metrics

### Lee el archivo CSV

In [2]:
# lee el catálogo
catalog = pd.read_csv('./datos_csv/1.root.csv')

# asegura que los impactos esten ordenados por CCD
#catalog = catalog.sort_values('muonCCD').reset_index(drop=True)
catalog

,eventNumber,muonCCD,centerX,centerY,sizeX,sizeY,asymX,asymY
0,15,1,-3701.897342,22563.553721,207.537701,212.857987,-31.277569,-32.923240
1,298,1,366.647121,-16626.957251,126.379226,140.235090,-17.894074,-6.171712
2,32,1,-6488.246795,20062.311898,152.315424,420.526559,-30.657469,-26.387688
3,39,1,7625.932138,-10653.649709,138.770036,138.782407,-24.422860,-23.320510
4,42,1,10167.331942,-21374.750786,189.139380,129.518107,32.010170,-4.301189
...,...,...,...,...,...,...,...,...
2697,4907,10,-10460.550482,-23120.142691,156.716007,154.565106,24.883776,-34.690667
2698,2506,10,-11062.662405,-29295.682293,150.598524,138.407647,20.062142,9.702173
2699,4405,10,-11558.039042,-29990.280551,155.430152,204.790065,-10.069407,35.313100
2700,4643,10,11941.330857,18188.333104,177.509099,165.867375,27.046667,-25.878916


### Algoritmo de tracking

In [ ]:
# parámetros del algoritmo
model = {
    'ccd_count': 10,
    'lr_short': {'slope':-18.36780106286495, 'intercept':6.881996494657647},
    'lr_long': {'slope':7.6418785841455525, 'intercept':-564.8448915083086},
    'asym_threshold': 18,
    'size_threshold': 175,
    'size_var': 200,
    'error_range': 500,
    'width_trace': 200,
}

In [3]:
# realiza una copia del catálogo sin los números de evento
reconstr = catalog.copy()
reconstr['eventNumber'] = 0
model['events_found'] = 0



# selecciona el impacto más similar entre los candidatos
def best_match(row, pred, model):
    
    # si hay cero o un candidato no se hace nada
    if len(pred.index) < 2:
        return pred
    
    sizeX = row['sizeX']
    sizeY = row['sizeY']
    
    asymX = row['asymX']
    asymY = row['asymY']

    # calcula una diferencia entre los tamaños y coeficientes de asimetría
    pred = pred.assign(size_diff = (pred['sizeX'] - sizeX)**2 + (pred['sizeY'] - sizeY)**2)
    pred = pred.assign(asym_diff = (pred['asymX'] - asymX)**2 + (pred['asymY'] - asymY)**2)
    pred = pred.assign(sort_metric = pred['size_diff'] + pred['asym_diff'])
    
    # selecciona el candidato con menor diferencia
    return pred.sort_values('sort_metric').drop(['size_diff', 'asym_diff', 'sort_metric'], axis=1).iloc[[0]]



# estima la posición del siguiente impacto y retorna los candidatos más cercanos
def filter_predictions(df, row, model):
            
    error = model['error_range']
    size_var = model['size_var']
    
    # estima el desplazamiento en el eje x
    if row['sizeX'] < model['size_threshold']:
        est_x = row['asymX']*model['lr_short']['slope'] + model['lr_short']['intercept']
    else:
        est_x = row['sizeX']*model['lr_long']['slope'] + model['lr_long']['intercept']
        if row['asymX'] > 0:
            est_x = -est_x
    
    # estima el desplazamiento en el eje y
    if row['sizeY'] < model['size_threshold']:
        est_y = row['asymY']*model['lr_short']['slope'] + model['lr_short']['intercept']
    else:
        est_y = row['sizeY']*model['lr_long']['slope'] + model['lr_long']['intercept']
        if row['asymY'] > 0:
            est_y = -est_y
    
    # suma el desplazamiento a la posición del impacto original
    est_x += row['centerX']
    est_y += row['centerY']

    # selecciona los impactos en la ventana de búsqueda y descarta aquellos muy diferentes
    prediction = df[(df['muonCCD'] == row['muonCCD']+1) & (
        ((est_x - error) < df['centerX']) & (df['centerX'] < (est_x + error))
    ) & (
        ((est_y - error) < df['centerY']) & (df['centerY'] < (est_y + error))
    ) & (
        ((row['sizeX'] - size_var) < df['sizeX']) & (df['sizeX'] < (row['sizeX'] + size_var))
    ) & (
        ((row['sizeY'] - size_var) < df['sizeY']) & (df['sizeY'] < (row['sizeY'] + size_var))
    )]
    
    # descarta los impactos ya visitados
    prediction = prediction[prediction['eventNumber'] == 0]
    return prediction



# traza una línea entre dos impactos y retorna los candidatos más cercanos
def trace_path(df, point_a, point_b, model):
    
    error = model['width_trace']
    size_var = model['size_var']
    
    sizeX = point_b['sizeX']
    sizeY = point_b['sizeY']

    # calcula la posición del siguiente impacto
    est_x = point_b['centerX'] + (point_b['centerX'] - point_a['centerX'])
    est_y = point_b['centerY'] + (point_b['centerY'] - point_a['centerY'])

    # selecciona los impactos en la ventana de búsqueda y descarta aquellos muy diferentes
    prediction = df[(df['muonCCD'] == point_b['muonCCD']+1) & (
        ((est_x - error) < df['centerX']) & (df['centerX'] < (est_x + error))
    ) & (
        ((est_y - error) < df['centerY']) & (df['centerY'] < (est_y + error))
    ) & (
        ((point_b['sizeX'] - size_var) < df['sizeX']) & (df['sizeX'] < (point_b['sizeX'] + size_var))
    ) & (
        ((point_b['sizeY'] - size_var) < df['sizeY']) & (df['sizeY'] < (point_b['sizeY'] + size_var))
    )]
    
    # descarta los impactos ya visitados
    prediction = prediction[prediction['eventNumber'] == 0]
    return prediction



# busca el siguiente impacto de un track
def follow_path(df, point_a, point_b, model, path):
    
    # si se llego al último CCD asigna un track a los impactos encontrados
    if point_b['muonCCD'] == model['ccd_count']:
        model['events_found'] += 1
        df.loc[path, 'eventNumber'] = model['events_found']
        return
    
    # obtiene los candidatos para el siguiente impacto
    pred = trace_path(df, point_a, point_b, model)
    pred = best_match(point_b, pred, model)
    
    # si no hay candidatos asigna un track a los impactos encontrados
    if pred.shape[0] == 0:
        model['events_found'] += 1
        df.loc[path, 'eventNumber'] = model['events_found']
        return
    
    # continúa trazando la línea
    path.append(pred.index[0])
    follow_path(df, point_b, pred.squeeze(), model, path)
    
#     for i, point_c in pred.iterrows():   
#         path.append(int(i))
#         follow_path(df, point_b, point_c, model, path)
#         path.pop()        
#         break



# itera a través de todos los impactos
for i in range(len(reconstr)):
    row = reconstr.iloc[i]
    
    # si el impacto ya fue visitado avanza al siguiente
    if row['eventNumber'] != 0:
        continue
    
    # determina cuales pueden ser los impactos siguientes y selecciona el más similar
    pred = filter_predictions(reconstr, row, model)
    pred = best_match(row, pred, model)
    
    # si no se encontró ningún impacto en el área de búsqueda
    if pred.shape[0] == 0:
        
        # asigna su propio track al impacto y avanza al siguiente
        model['events_found'] += 1
        reconstr.loc[i, 'eventNumber'] = model['events_found']
        continue
    
    # agrega los impactos hallados al track
    path = [i, pred.index[0]]
    
    # traza una línea en busca de los impactos siguientes
    follow_path(reconstr, row, pred.squeeze(), model, path)

#     path = [i]
#     for j, point in pred.iterrows():
#         path.append(int(j))
#         follow_path(reconstr, row, point, model, path)
#         path.pop()
#         break

# calcula la métrica de desempeño
metrics.adjusted_mutual_info_score(catalog['eventNumber'], reconstr['eventNumber'])

0.9997052567093271